# Segmented Representations

One common representation in evolutionary algorithms (EA) is that of a "segmented representation."  That is, each individual is comprised of a sequence of segments, which are themselves fixed-length sequences, and are usually binary, but needn't be.  Each segment represents a salient feature, such as a rule in a Pitt Approach system, or a convolutional layer and its hyperparameters, as is the case for Multi-node Evolutionary Neural Networks for Deep Learning (MENNDL).

There are two broad categories for these systems: those that have a fixed number of such segments, as is the case currently for MENNDL, and a dynamic number of segments, as is the case for Pitt Approach classifiers.

In this notebook we look at LEAP support for segmented representations, starting with initializers and decoders, and then looking at the mutation pipeline operator.  We then plug all that into a simple EA example.


In [1]:
import sys
import random
import functools
import pprint

from leap_ec.individual import Individual

from leap_ec.segmented_rep.initializers import create_segmented_sequence
from leap_ec.segmented_rep.decoders import SegmentedDecoder
from leap_ec.segmented_rep.ops import apply_mutation

from leap_ec.binary_rep.initializers import create_binary_sequence
from leap_ec.binary_rep.ops import perform_mutate_bitflip
from leap_ec.binary_rep.decoders import BinaryToIntDecoder

from leap_ec.real_rep.initializers import create_real_vector
from leap_ec.real_rep.ops import mutate_gaussian

## Binary genomes

We first look at segmented representations with segments that use a binary representaion.

In [2]:
# Create a genome of four segments of five binary digits.
seg = create_segmented_sequence(4, create_binary_sequence(5))
print(seg)

[[0, 1, 1, 0, 1], [1, 0, 1, 0, 1], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0]]


In [3]:
# Now create five genomes of varying length by passing in a function for `length` that provides an
# integer drawn from a distribution.
seqs = [] # Save sequences for next step
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=5), create_binary_sequence(5))
    print(i, seq)
    seqs.append(seq)

0 [[0, 0, 1, 0, 1], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0]]
1 [[0, 0, 0, 0, 1], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]
2 [[0, 0, 1, 0, 1], [1, 0, 0, 0, 1], [0, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]
3 [[1, 0, 0, 0, 0], [0, 0, 0, 1, 0]]
4 [[0, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 0, 1], [1, 0, 1, 1, 1]]


Now let's see about decoding those segments.  The segmented representation relies on a secondary decoder that's applied to each segment.  In this case, we'll just use a simple binary to int decoder on the segments we created in the previous step.

In [4]:
# We want each segment to have two integers from the five bits.
decoder = SegmentedDecoder(BinaryToIntDecoder(2,3)) 

for i, seq in enumerate(seqs):
    vals = decoder.decode(seq)
    print(i, vals)

0 [[0, 5], [2, 0], [1, 0]]
1 [[0, 1], [1, 2], [3, 1]]
2 [[0, 5], [2, 1], [0, 6], [2, 2], [2, 3]]
3 [[2, 0], [0, 2]]
4 [[1, 4], [0, 5], [0, 5], [2, 7]]


In [5]:
# And now for mutation, which shows that, on average, a single value is changed in an example individual.  The
# takeaway here is that segmented mutation just uses a mutator from another representation and naively applies it.

original = Individual([[0,0],[1,1]])
print('original:', original)
mutated = next(apply_mutation(iter([original]),mutator=perform_mutate_bitflip))
print('mutated:', mutated)

original: [[0, 0], [1, 1]]
mutated: [[0, 0], [0, 1]]


## Real-valued genomes

Now we demonstrate the same process using a real-valued representation.

In [6]:
# Create five segmented sequences that vary from 1 to 3 segments
bounds = ((-5.12,5.12), (-1,1), (-10,10)) # three reals and their respective bounds for sampling
seqs = []
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=3), 
                                    create_real_vector(bounds))
    seqs.append(seq)

# Just for fun, now add a genome that has exactly 5 segments
seqs.append(create_segmented_sequence(5, create_real_vector(bounds)))

for i, s in enumerate(seqs):
    print(i, pprint.pformat(s, indent=2))

0 [ [-2.579245798740983, 0.8914407602776055, 8.661978797568114],
  [4.446234438366585, -0.524455978960273, 7.715610812421872],
  [1.767512721627309, 0.9435481470222562, -6.791288472437844]]
1 [ [3.3098591428580635, 0.8011705498003272, 3.824976154678726],
  [2.806247363277361, -0.5153472396385919, -4.026898963617289],
  [3.1362591334882515, 0.28186441259480377, -1.9870733662102218]]
2 [ [0.6750098455707008, -0.7549709096991539, -9.835613455086438],
  [-0.2605899678832566, 0.12391459462027954, 4.709111142220257]]
3 [ [-3.601422937293163, -0.5780658288887797, -9.588512530150293],
  [4.9170505329262495, -0.031223540097975233, -7.821213383159213]]
4 [[-1.9099967466778072, -0.6529909163598429, 4.263146718634292]]
5 [ [-1.5422431554437184, -0.1441865990132567, -4.454070686653897],
  [-0.863347940026471, -0.39074823628984223, -1.3065430020844992],
  [4.649852330445722, 0.32868504618425143, 6.44137991870538],
  [2.4378031856066373, -0.23644628448726035, -1.0554999598501276],
  [-1.5774252388158

Now we repeat the application of the segmented mutation operator, but this time to real-valued genomes.

In [7]:
original = Individual([[0.0,0.0],[1.0,1.0],[-1.0,0.0]])
print('original:', original)
mutated = next(apply_mutation(iter([original]),mutator=perform_mutate_bitflip))
print('mutated:', mutated)

original: [[0.0, 0.0], [1.0, 1.0], [-1.0, 0.0]]
mutated: [[1.0, 0.0], [1.0, 1.0], [-1.0, 0.0]]
